In [1]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [2]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [4]:
import cookin_ink

reload(cookin_ink)


# DEFAULT_PARAMS = {
#     'ink_window_size': 20,
#     'ink_take_threshold': 1.0,
#     'ink_clear_threshold': 0.5,
# }

param_grid = {
    'ink_window_size': [10, 15, 20, 25, 30, 40, 50],
    'ink_take_threshold': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0],
    'ink_clear_threshold': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0],
}


param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    trader = cookin_ink.Trader(params)
    days = [1]
    profits = backtest_days(trader, days=days, data_path='.', round_num=4)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

100%|██████████| 378/378 [11:35<00:00,  1.84s/it]


In [5]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,ink_window_size,ink_take_threshold,ink_clear_threshold,profit_day_1,tot_profit
372,50,5.0,0.5,1695.0,1695.0
366,50,4.0,0.5,1242.0,1242.0
318,40,5.0,0.5,993.0,993.0
373,50,5.0,1.0,803.0,803.0
156,20,5.0,0.5,572.0,572.0
...,...,...,...,...,...
11,10,1.0,3.0,-158196.0,-158196.0
10,10,1.0,2.5,-158196.0,-158196.0
9,10,1.0,2.0,-158196.0,-158196.0
8,10,1.0,1.5,-158196.0,-158196.0


In [8]:
df

,sma_window_size,std_window_size,thresh_z,profit_day_-1,tot_profit
410,300,60,4.0,7642.0,7642.0
385,300,25,6.5,3615.0,3615.0
413,300,60,5.5,2705.0,2705.0
428,300,80,5.5,2622.0,2622.0
427,300,80,5.0,2135.0,2135.0
...,...,...,...,...,...
0,30,25,1.5,-222360.0,-222360.0
30,30,60,1.5,-225165.0,-225165.0
45,30,80,1.5,-225165.0,-225165.0
15,30,30,1.5,-225165.0,-225165.0


In [12]:
profits

[0.0]

In [ ]:
df.to_csv('backtests/inktrader_bt_results.csv')

,ink_change_threshold_pct,ink_window_size,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.02,5,4273.0,3450.0,0.0,7723.0
1,0.02,10,4121.0,3450.0,0.0,7571.0
2,0.02,15,4121.0,3450.0,0.0,7571.0
3,0.03,5,0.0,3450.0,0.0,3450.0
4,0.03,10,0.0,3450.0,0.0,3450.0
5,0.03,15,0.0,3450.0,0.0,3450.0
6,0.04,5,0.0,3450.0,0.0,3450.0
7,0.04,10,0.0,3450.0,0.0,3450.0
8,0.04,15,0.0,3450.0,0.0,3450.0
